In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import yaml
import json
from collections import defaultdict
from pandas import json_normalize

from config_valid import Config

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.height', 500)

# Dal file di config in yaml a pandas dataframe per riempire le variabili dei trials e di nuovo indietro a liste di dizionari per le configs

In [29]:
config_file = "configurations/classification_cm.yml"
config_file2 = "configurations/classification_grid.yml"
config_c = Config(config_file)

### Carico il file yaml e lo trasformo in dataframe con json_normalize

In [30]:
conf = yaml.safe_load(open(config_file))
conf2 = yaml.safe_load(open(config_file2))
yaml_df = json_normalize(conf)
yaml_df2 = json_normalize(conf2)

In [10]:
yaml_df

,device,graph_dataset.ERmodel,graph_dataset.regular,graph_dataset.confmodel,graph_dataset.continuous_p,graph_dataset.list_p,graph_dataset.range_p,graph_dataset.Num_nodes,graph_dataset.Num_grafi_per_tipo,graph_dataset.Num_grafi_totali,graph_dataset.list_degree,graph_dataset.random_node_feat,graph_dataset.list_exponents,training.mode,training.learning_rate,training.epochs,training.batch_size,training.percentage_train,training.earlystop_patience,training.epochs_model_checkpoint,training.every_epoch_embedding,model.last_layer_dense,model.GCNneurons_per_layer,model.neurons_last_linear,model.freezeGCNlayers,model.autoencoder,model.put_batchnorm,model.put_dropout,model.final_pool_aggregator,model.node_features_dim,logging.train_step_print
0,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[1, 16, 16, 1]","[3, 3, 1]",False,False,False,False,True,1,100


In [20]:
yaml_df2

,device,graph_dataset.ERmodel,graph_dataset.regular,graph_dataset.continuous_p,graph_dataset.list_p,graph_dataset.range_p,graph_dataset.Num_nodes,graph_dataset.Num_grafi_per_tipo,graph_dataset.Num_grafi_totali,graph_dataset.list_degree,training.mode,training.learning_rate,training.epochs,training.batch_size,training.percentage_train,training.earlystop_patience,model.last_layer_dense,model.neurons_per_layer,model.autoencoder,model.put_batchnorm,model.node_features_dim,logging.train_step_print
0,gpu,True,False,False,"[0.05, 0.3]","[0.05, 0.6]",300,300,1000,"[4, 50]",mode1,0.01,58500,150,0.7,100,False,"[1, 8, 2]",False,False,1,1000


### Moltiplico le righe per le prove di ciascuna variabile

In [31]:
# moltiplico le righe per il numero di prove del trial
def df_singlevariable_trials(conf_df, trial_list, multicolums):
    nrows = len(trial_list)
    repeated = pd.DataFrame(np.repeat(conf_df.values, nrows, axis=0), columns=conf_df.columns)
    repeated.loc[:, multicolums] = trial_list
    return repeated

In [32]:
trials_numgrafi = [200, 150, 100, 50]
var1 = 'graph_dataset.Num_grafi_per_tipo'
df1 = df_singlevariable_trials(yaml_df, trials_numgrafi, var1)
df1

,device,graph_dataset.ERmodel,graph_dataset.regular,graph_dataset.confmodel,graph_dataset.continuous_p,graph_dataset.list_p,graph_dataset.range_p,graph_dataset.Num_nodes,graph_dataset.Num_grafi_per_tipo,graph_dataset.Num_grafi_totali,graph_dataset.list_degree,graph_dataset.random_node_feat,graph_dataset.list_exponents,training.mode,training.learning_rate,training.epochs,training.batch_size,training.percentage_train,training.earlystop_patience,training.epochs_model_checkpoint,training.every_epoch_embedding,model.last_layer_dense,model.GCNneurons_per_layer,model.neurons_last_linear,model.freezeGCNlayers,model.autoencoder,model.put_batchnorm,model.put_dropout,model.final_pool_aggregator,model.node_features_dim,logging.train_step_print
0,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[1, 16, 16, 1]","[3, 3, 1]",False,False,False,False,True,1,100
1,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",150,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[1, 16, 16, 1]","[3, 3, 1]",False,False,False,False,True,1,100
2,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",100,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[1, 16, 16, 1]","[3, 3, 1]",False,False,False,False,True,1,100
3,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",50,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[1, 16, 16, 1]","[3, 3, 1]",False,False,False,False,True,1,100


In [33]:
# provo con un altro trials e moltiplico con OUTER
trials_layers = [ [1, 16, 16, 1], 
                 [1, 8, 8, 1],
                 [1, 4, 4, 1],
                 [1, 2, 2, 1]]
var2 = 'model.GCNneurons_per_layer'
#df2 = df_singlevariable_trials(yaml_df, [tuple(t) for t in trials_layers], var2)
df2 = pd.DataFrame(columns=[var2])
df2[var2] = trials_layers#  [tuple(t) for t in trials_layers]
df2

,model.GCNneurons_per_layer
0,"[1, 16, 16, 1]"
1,"[1, 8, 8, 1]"
2,"[1, 4, 4, 1]"
3,"[1, 2, 2, 1]"


### Outer product dei dataframe in modo incrementale

In [34]:
#devo togliere dal precedente df la colonna di variabili che vengono cambiate dal successivo df
df1.drop(var2, axis=1, inplace=True)

In [14]:
df2

,model.GCNneurons_per_layer
0,"[1, 16, 16, 1]"
1,"[1, 8, 8, 1]"
2,"[1, 4, 4, 1]"
3,"[1, 2, 2, 1]"


In [10]:
#othercols = df2.columns.difference(df1.columns)
#othercols

In [35]:
def list2tuples(df):
    cols2convert = (df.applymap(type) == list).all().index.to_list()
    for col in cols2convert:
        df[col] = df[col].apply(tuple)

In [36]:
s = (df1.applymap(type) == list).all()#.index.to_list()
s[s]

graph_dataset.list_p                True
graph_dataset.range_p               True
graph_dataset.Num_nodes             True
graph_dataset.list_degree           True
graph_dataset.list_exponents        True
training.epochs_model_checkpoint    True
model.neurons_last_linear           True
dtype: bool

In [37]:
list2tuples(df2)

In [38]:
#res = pd.merge(df1, df2, on=df1.columns.tolist(), how='outer')
res = pd.merge(df1, df2, how='cross')

In [39]:
res

,device,graph_dataset.ERmodel,graph_dataset.regular,graph_dataset.confmodel,graph_dataset.continuous_p,graph_dataset.list_p,graph_dataset.range_p,graph_dataset.Num_nodes,graph_dataset.Num_grafi_per_tipo,graph_dataset.Num_grafi_totali,graph_dataset.list_degree,graph_dataset.random_node_feat,graph_dataset.list_exponents,training.mode,training.learning_rate,training.epochs,training.batch_size,training.percentage_train,training.earlystop_patience,training.epochs_model_checkpoint,training.every_epoch_embedding,model.last_layer_dense,model.neurons_last_linear,model.freezeGCNlayers,model.autoencoder,model.put_batchnorm,model.put_dropout,model.final_pool_aggregator,model.node_features_dim,logging.train_step_print,model.GCNneurons_per_layer
0,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 16, 16, 1)"
1,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 8, 8, 1)"
2,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 4, 4, 1)"
3,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 2, 2, 1)"
4,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",150,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 16, 16, 1)"
5,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",150,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 8, 8, 1)"
6,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",150,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 4, 4, 1)"
7,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",150,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 2, 2, 1)"
8,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",100,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 16, 16, 1)"
9,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",100,400,"[4, 50]",False,"[-2.5, -4.5]",mode2,0.1,500,100,0.7,50,[-1],True,False,"[3, 3, 1]",False,False,False,False,True,1,100,"(1, 8, 8, 1)"


### Riprendo la gerarchia del file yaml

In [40]:
res.columns = res.columns.str.split('.', expand=True)
res

device graph_dataset                                              \
      NaN       ERmodel regular confmodel continuous_p       list_p   
0     gpu         False   False      True         True  [0.05, 0.3]   
1     gpu         False   False      True         True  [0.05, 0.3]   
2     gpu         False   False      True         True  [0.05, 0.3]   
3     gpu         False   False      True         True  [0.05, 0.3]   
4     gpu         False   False      True         True  [0.05, 0.3]   
5     gpu         False   False      True         True  [0.05, 0.3]   
6     gpu         False   False      True         True  [0.05, 0.3]   
7     gpu         False   False      True         True  [0.05, 0.3]   
8     gpu         False   False      True         True  [0.05, 0.3]   
9     gpu         False   False      True         True  [0.05, 0.3]   
10    gpu         False   False      True         True  [0.05, 0.3]   
11    gpu         False   False      True         True  [0.05, 0.3]   
12    gpu         False   False      True         True  [0.05, 0.3]   
13    gpu         False   False      True         True  [0.05, 0.3]   
14    gpu         False   False      True         True  [0.05, 0.3]   
15    gpu         False   False      True         True  [0.05, 0.3]   

                                                                             \
        range_p   Num_nodes Num_grafi_per_tipo Num_grafi_totali list_degree   
0   [0.05, 0.6]  [700, 700]                200              400     [4, 50]   
1   [0.05, 0.6]  [700, 700]                200              400     [4, 50]   
2   [0.05, 0.6]  [700, 700]                200              400     [4, 50]   
3   [0.05, 0.6]  [700, 700]                200              400     [4, 50]   
4   [0.05, 0.6]  [700, 700]                150              400     [4, 50]   
5   [0.05, 0.6]  [700, 700]                150              400     [4, 50]   
6   [0.05, 0.6]  [700, 700]                150              400     [4, 50]   
7   [0.05, 0.6]  [700, 700]                150              400     [4, 50]   
8   [0.05, 0.6]  [700, 700]                100              400     [4, 50]   
9   [0.05, 0.6]  [700, 700]                100              400     [4, 50]   
10  [0.05, 0.6]  [700, 700]                100              400     [4, 50]   
11  [0.05, 0.6]  [700, 700]                100              400     [4, 50]   
12  [0.05, 0.6]  [700, 700]                 50              400     [4, 50]   
13  [0.05, 0.6]  [700, 700]                 50              400     [4, 50]   
14  [0.05, 0.6]  [700, 700]                 50              400     [4, 50]   
15  [0.05, 0.6]  [700, 700]                 50              400     [4, 50]   

                                   training                                  \
   random_node_feat list_exponents     mode learning_rate epochs batch_size   
0             False   [-2.5, -4.5]    mode2           0.1    500        100   
1             False   [-2.5, -4.5]    mode2           0.1    500        100   
2             False   [-2.5, -4.5]    mode2           0.1    500        100   
3             False   [-2.5, -4.5]    mode2           0.1    500        100   
4             False   [-2.5, -4.5]    mode2           0.1    500        100   
5             False   [-2.5, -4.5]    mode2           0.1    500        100   
6             False   [-2.5, -4.5]    mode2           0.1    500        100   
7             False   [-2.5, -4.5]    mode2           0.1    500        100   
8             False   [-2.5, -4.5]    mode2           0.1    500        100   
9             False   [-2.5, -4.5]    mode2           0.1    500        100   
10            False   [-2.5, -4.5]    mode2           0.1    500        100   
11            False   [-2.5, -4.5]    mode2           0.1    500        100   
12            False   [-2.5, -4.5]    mode2           0.1    500        100   
13            False   [-2.5, -4.5]    mode2           0.1    500        100   
14            False   [-2.5, -4.5]   

### Pivot del multindex dalle colonne alle righe

In [68]:
#for i in range(res.shape[0]):
#    display(res.iloc[i:i+1])

#for each row
row = res.iloc[0:1]
row

device graph_dataset                                              \
     NaN       ERmodel regular confmodel continuous_p       list_p   
0    gpu         False   False      True         True  [0.05, 0.3]   

                                                                            \
       range_p   Num_nodes Num_grafi_per_tipo Num_grafi_totali list_degree   
0  [0.05, 0.6]  [700, 700]                200              400     [4, 50]   

                                  training                                  \
  random_node_feat list_exponents     mode learning_rate epochs batch_size   
0            False   [-2.5, -4.5]    mode2           0.1    500        100   

                                                               \
  percentage_train earlystop_patience epochs_model_checkpoint   
0              0.7                 50                    [-1]   

                                   model                                      \
  every_epoch_embedding last_layer_dense neurons_last_linear freezeGCNlayers   
0                  True            False           [3, 3, 1]           False   

                                                               \
  autoencoder put_batchnorm put_dropout final_pool_aggregator   
0       False         False       False                  True   

                             logging                model  
  node_features_dim train_step_print GCNneurons_per_layer  
0                 1              100       (1, 16, 16, 1)

In [71]:
p = pd.DataFrame(row.stack(level=[0,1], dropna=True)).reset_index(0).drop('level_0', axis=1)

In [72]:
p

0
graph_dataset ERmodel                           False
              Num_grafi_per_tipo                200.0
              Num_grafi_totali                    400
              Num_nodes                    [700, 700]
              confmodel                          True
              continuous_p                       True
              list_degree                     [4, 50]
              list_exponents             [-2.5, -4.5]
              list_p                      [0.05, 0.3]
              random_node_feat                  False
              range_p                     [0.05, 0.6]
              regular                           False
logging       train_step_print                    100
model         GCNneurons_per_layer     (1, 16, 16, 1)
              autoencoder                       False
              final_pool_aggregator              True
              freezeGCNlayers                   False
              last_layer_dense                  False
              neurons_last_linear           [3, 3, 1]
              node_features_dim                     1
              put_batchnorm                     False
              put_dropout                       False
training      batch_size                          100
              earlystop_patience                   50
              epochs                              500
              epochs_model_checkpoint            [-1]
              every_epoch_embedding              True
              learning_rate                       0.1
              mode                              mode2
              percentage_train                    0.7

In [31]:
stacked = pd.DataFrame(row.stack(level=[0,1], dropna=True)[0])#.set_index(['graph_dataset', 'training', 'model'])
stacked

AttributeError: 'Series' object has no attribute 'stack'

In [32]:
# funzione per gestire il dataframe in formato dizionario e prepararlo al json
def nested_dict():
    return defaultdict(nested_dict)
result = nested_dict()

### ottengo un dizionario pronto da salvare in stringa json

In [33]:
for r in stacked.itertuples():
    result[r.Index[0]][r.Index[1]] = r._1
    #print(row.Index)
result

defaultdict(<function __main__.nested_dict()>,
            {'graph_dataset': defaultdict(<function __main__.nested_dict()>,
                         {'ERmodel': False,
                          'Num_grafi_per_tipo': 200.0,
                          'Num_grafi_totali': 400,
                          'Num_nodes': [700, 700],
                          'confmodel': True,
                          'continuous_p': True,
                          'list_degree': [4, 50],
                          'list_exponents': [-2.5, -4.5],
                          'list_p': [0.05, 0.3],
                          'random_node_feat': False,
                          'range_p': [0.05, 0.6],
                          'regular': False}),
             'logging': defaultdict(<function __main__.nested_dict()>,
                         {'train_step_print': 100}),
             'model': defaultdict(<function __main__.nested_dict()>,
                         {'GCNneurons_per_layer': (1, 16, 16, 1),
                 

In [34]:
config_is_back = yaml.safe_load(json.dumps(result))
config_is_back

NameError: name 'result' is not defined

In [35]:
# il livello 1 (dove c'è solo device senza livello 2) manca, al momento mi accontento di gestire solo con 2 livelli, no di più e non di meno

In [36]:
config_is_back['device'] = 'gpu'

In [37]:
conf == config_is_back

True

# Test classe GridConfigurations

In [2]:
from GridConfigurations import GridConfigurations

In [78]:
config_file = "configurations/classification_cm.yml"
config_class = Config(config_file)

diz_trials = {'model.GCNneurons_per_layer' : [ [1, 16, 16, 1], 
                                             [1, 8, 8, 1],
                                             [1, 4, 4, 1],
                                             [1, 2, 2, 1]],
              'graph_dataset.Num_grafi_per_tipo': [200, 150, 100, 50] }

gc = GridConfigurations(config_class, diz_trials)
gc.make_configs()

In [80]:
len(gc.configs)

16

In [81]:
gc.config_dataframe

device graph_dataset                                              \
      NaN       ERmodel regular confmodel continuous_p       list_p   
0     gpu         False   False      True         True  [0.05, 0.3]   
1     gpu         False   False      True         True  [0.05, 0.3]   
2     gpu         False   False      True         True  [0.05, 0.3]   
3     gpu         False   False      True         True  [0.05, 0.3]   
4     gpu         False   False      True         True  [0.05, 0.3]   
5     gpu         False   False      True         True  [0.05, 0.3]   
6     gpu         False   False      True         True  [0.05, 0.3]   
7     gpu         False   False      True         True  [0.05, 0.3]   
8     gpu         False   False      True         True  [0.05, 0.3]   
9     gpu         False   False      True         True  [0.05, 0.3]   
10    gpu         False   False      True         True  [0.05, 0.3]   
11    gpu         False   False      True         True  [0.05, 0.3]   
12    gpu         False   False      True         True  [0.05, 0.3]   
13    gpu         False   False      True         True  [0.05, 0.3]   
14    gpu         False   False      True         True  [0.05, 0.3]   
15    gpu         False   False      True         True  [0.05, 0.3]   

                                                                           \
        range_p   Num_nodes Num_grafi_totali list_degree random_node_feat   
0   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
1   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
2   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
3   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
4   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
5   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
6   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
7   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
8   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
9   [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
10  [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
11  [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
12  [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
13  [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
14  [0.05, 0.6]  [700, 700]              400     [4, 50]            False   
15  [0.05, 0.6]  [700, 700]              400     [4, 50]            False   

                  training                                                   \
   list_exponents     mode learning_rate epochs batch_size percentage_train   
0    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
1    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
2    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
3    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
4    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
5    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
6    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
7    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
8    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
9    [-2.5, -4.5]    mode2           0.1    500        100              0.7   
10   [-2.5, -4.5]    mode2           0.1    500        100              0.7   
11   [-2.5, -4.5]    mode2           0.1    500        100              0.7   
12   [-2.5, -4.5]    mode2           0.1    500        100              0.7   
13   [-2.5, -4.5]    mode2           0.1    500        100              0.7   
14   [-2.5, -4.5]    mode2           0.1    500        100              0

# quando aggiungerò i risultati del training al dataframe

In [ ]:
yaml_df.at[i, ('risultati','testloss')] = 1

### concateno i nuovi risultati al dataframe esistente con i risultati finora accumulati

In [16]:
df_tot = pd.concat([yaml_df, yaml_df2])


,device,graph_dataset.ERmodel,graph_dataset.regular,graph_dataset.confmodel,graph_dataset.continuous_p,graph_dataset.list_p,graph_dataset.range_p,graph_dataset.Num_nodes,graph_dataset.Num_grafi_per_tipo,graph_dataset.Num_grafi_totali,...,model.GCNneurons_per_layer,model.neurons_last_linear,model.freezeGCNlayers,model.autoencoder,model.put_batchnorm,model.put_dropout,model.final_pool_aggregator,model.node_features_dim,logging.train_step_print,model.neurons_per_layer
0,gpu,False,False,True,True,"[0.05, 0.3]","[0.05, 0.6]","[700, 700]",200,400,...,"[1, 16, 16, 1]","[3, 3, 1]",False,False,False,False,True,1,100,NaN
0,gpu,True,False,NaN,False,"[0.05, 0.3]","[0.05, 0.6]",300,300,1000,...,NaN,NaN,NaN,False,False,NaN,NaN,1,1000,"[1, 8, 2]"
